# Monte Carlo Wealth

The goal of this notebook is to explore the equivalence between analysing return distribution parameters and analysing a posteriori parameters of a wealth distribution generted through Monte Carlo simulation.

Throughout this series we will normally be using log returns instead of returns as they have a series of interesting properties like being more normally distributed than returns or not allowing stock prices to go negative. They are calculated like
$$ r_i=ln(1+R_i) $$

Some of their interesting properties is that they are additive over time. That is,
$$ r_{1,3} = r_{1,2}+r_{2,3} $$
instead of
$$ R_{1,3} = (1+R_{1,2})(1+R_{2,3})-1 $$
The proof can be seen by adding 1 to both sides and taking logarithms. Log returns are however not additive over assets. That is
$$ R_i^{(P)} = \sum_{k}w_i^{(k)} R_i^{(k)} $$
$$ r_i^{(P)} = \ln{\sum_k w^{(k)} e^{r_i^{(k)}}}$$


This last bit is however not important right now, as we are interested in calculating the statistics of a single asset. We are interested in those statistics as related to wealth instead of returns, as that allows for more interesting and intuitive constraints in future experiments.
Statistics on wealth will be calculated by simulating the progress of our asset over time across many different potential scenarios, thanks to Monte Carlo simulation.

Some theoretical basis for this equivalence. We have
$$ r_i \sim \mathcal{N}(\mu,\sigma^2) $$
$$ W_T = W_0 e^{\sum_{t=1}^{T}{r_t}} $$
Thus the equivalence should be
$$ \ln{\frac{W_T}{W_0}} \sim \mathcal{N}(T\mu,T \sigma^2) $$

In [17]:
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.stats import norm

In [14]:
W0 = 1000
MU = 0.05
STD = 0.02
T = 10
N_SCENARIOS = 100000

In [15]:
r_matrix = np.random.normal(loc=MU,scale=STD,size=(N_SCENARIOS,T))
r_total_returns = np.sum(r_matrix,axis=1)
W_T = W0 * np.exp(r_total_returns)
W_T_adj = np.log(W_T/W0)
numerical_mu = np.average(W_T_adj)
numerical_std = np.std(W_T_adj)
theoretical_mu = MU*T
theoretical_std = np.sqrt(T*STD**2)
print(f"MU: Theoretical: {theoretical_mu:.4f}, Empirical: {numerical_mu:.4f}, Error: {(numerical_mu/theoretical_mu-1)*100:.2f}%")
print(f"STD: Theoretical: {theoretical_std:.4f}, Empirical: {numerical_std:.4f}, Error: {(numerical_std/theoretical_std-1)*100:.2f}%")

MU: Theoretical: 0.5000, Empirical: 0.4999, Error: -0.02%
STD: Theoretical: 0.0632, Empirical: 0.0631, Error: -0.17%


In [42]:
num_y, num_bins = np.histogram(W_T_adj,bins=int(N_SCENARIOS**(1/2)),density=True)
num_x = (num_bins + np.roll(num_bins,1))/2
num_x = num_x[1:]
theo_x = np.linspace(theoretical_mu - 5*theoretical_std, theoretical_mu + 5*theoretical_std, 100)
theo_y = norm.pdf(theo_x, loc=theoretical_mu, scale=theoretical_std)
fig = go.Figure()
fig.add_trace(go.Scatter(x=num_x,y=num_y,mode='lines',name='numerical'))
fig.add_trace(go.Scatter(x=theo_x,y=theo_y,mode='lines',name='theoretical'))
fig.show()